<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT22/blob/filippaolofsson-lab2/Lab2/filippaolofsson_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab: matrix factorization**
**Filippa Olofsson**

# **Abstract**

This lab consists of three problems. The problems being investigated in this report are sparse matrix-vector product, QR factorization and direct solver.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [ ]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [ ]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common 
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *
    
#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

The problems being investigated in this report are sparse matrix-vector product, QR factorization and direct solver.



# **Method**

I used some of the functions made in Lab: Introduction. 

In [ ]:
# Imported code fram Laboration: Introduction

def scalar_product(x, y):
  scalar = 0
  for i in range(len(x)):
    scalar += x[i]*y[i]
  return scalar

def norm(x):
  x = scalar_product(x, x)
  e_norm = x**(1/2)
  return e_norm

def matrix_vector_product(x, A):
  m, n = np.shape(A)  # row, column
  b = np.zeros((m, 1))
  for i in range(m): 
    sum = 0
    for j in range(n):
      sum += A[i][j]*x[j]
    b[i] = sum
  return b

def matrix_matrix_product(A, B):
  m_a, n_a = np.shape(A)
  m_b, n_b = np.shape(B)
  C = np.zeros((m_a, n_b))
  if n_a != m_b:
    return "No answer"
  else:
    for i in range(m_a): # for each row in A
      for j in range(n_b): # for each column i B
        sum = 0
        for k in range(n_a): # for each column in A
          sum += A[i][k] * B[k][j]
        C[i][j] = sum
    return C

def euclidian_dist(x, y):
  if len(x) != len(y):
    return "No answer"
  else:
    sum_dist = 0
    for i in range(len(x)):
      sum_dist += (x[i] - y[i])**2
    e_dist = sum_dist**(1/2)
    return e_dist

### **1. Function: sparse matrix-vector product**

The sparse matrix-vector product is calculated thorugh Algorithm 5.9, Chapter 5. By first creating a vector b, which has the same size as the vector x. The outer for-loop is iterating over the number of rows n, and the inner for-loop can be seen iterating over the non-zero elements in each row. Each non-zero element is multiplied by corresponding element in the x-vector and added to the final element in b. 

The results are compared with the dense matrix-vector product. The function test_svmp builds the matrix A from the given vectors and calculates the dense matrix-vector product.   

In [ ]:
# Input: vector x, sparse (real, quadratic) matrix A: CRS arrays val, col_idx, row_ptr
# Output: matrix-vector product b=Ax

def sparse_matrix_vector_product(x, val, col_idx, row_ptr):
  b = np.zeros(x.shape)
  n = len(b)
  for i in range(0, n):
    for j in range(row_ptr[i]-1, row_ptr[i+1]-1):
      b[i]= b[i] + val[j]*x[col_idx[j]-1]
  return b

# Creating A from val, col_idx, row_ptr
def test_smvp(x, val, col_idx, row_ptr):
  A = np.zeros((len(x), len(x)))
  num_rows = len(row_ptr) - 1
  for i in range(num_rows):
    num_columns = row_ptr[i+1] - row_ptr[i]
    row_start = row_ptr[i] -1
    slice_columns = col_idx[row_start:row_start + num_columns]
    slice_val = val[row_start:row_start + num_columns]
    k = 0
    for j in slice_columns:
      A[i][j-1] = slice_val[k]
      k += 1
  # Calculate the dense matrix-vector product
  b = matrix_vector_product(x, A)
  return b


### **2. Function: QR factorization**

The QR factorization is calculated thorugh algorithm 5.3 (modified Gram Schmidt), Chapter 5.

The frobenius norms is not calculated since both $Q^TQ - I$ and $QR - A$ should equal a matrix of zeros if the QR factorization was made successfully. Therefore, the function "frobenius_norms" is only checking if the equations result in a zero-matrix.

In [ ]:
from numpy.ma.core import shape
# Input: (real quadratic) matrix A
# Output: orthogonal matrix Q, upper triangular matrix R, such that A=QR

def QR_fac(A):
  n, n = np.shape(A)
  v = np.zeros(shape=(1, n))
  R = np.zeros((n,n))
  Q = np.zeros((n,n))
  for j in range(0, n):
    v[:] = A[:,j]
    for i in range(0, j):
      R[i,j] = scalar_product(Q[:,i], np.transpose(v[:]))
      v[:] = v[:] - R[i,j]*Q[:,i]
    R[j,j] = norm(np.transpose(v))
    Q[:,j] = (v[:]/R[j,j])
  return Q, R

# Not actually calculating frobenius norm, just see if F equals a zero-matrix
def frobenius_norms(A, B, C): 
  D = matrix_matrix_product(A, B)
  F = D - C 
  if F.all() == 0:
    return 0
  else:
    return 1

### **3. Function: direct solver Ax=b**

The problem is solved by using the QR factorization of matrix A. With the QR factorization the problem can be rewritten as:

$Ax = b \longrightarrow QRx=b \longrightarrow  Rx = Q^{-1}b \longrightarrow$ (Let $y = Q^{-1}b$) $\longrightarrow Rx = y \longrightarrow x = R^{-1}y$

The direct solver function uses QR_fac to factorize A into Q and R. Since Q is an orthogonal matrix $Q^{-1} = Q^T$. Using the $Q^T$b to calculate y. Then using Algorithm 5.2 (Backward substitution) Chapter 5 to solve the equation $x = R^{-1}y$. Returning x.

The backward substitution function is used since R is a upper triangular matrix. The function starts from the last component of the solution vector x. The inner loop is a scalar product between vector x and the i:th row of the matrix R. (Chapter 5, p.87)

In [ ]:
# Input: (real, quadratic) matrix A, vector b
# Output: vector x=A^(-1)b

def direct_solver(A, b):
  Q, R = QR_fac(A)
  Q1 = np.transpose(Q)
  y = matrix_vector_product(b, Q1)
  x = backward_substitution(R, y)
  return x

def backward_substitution(R, y):
  n = len(y)
  x = np.zeros(shape=(n, 1))
  x[n-1] = y[n-1]/R[n-1,n-1]
  for i in range(n-2, -1, -1):
    sum = 0
    for j in range(i, n):
      sum = sum + R[i,j]*x[j]
    x[i] = (y[i] - sum)/R[i,i]
  return x

# **Results**

The tests of each function is made below. All tests made should pass and are further described in the code below.

In [ ]:
# Tests 1: sparce matrix-vector product 
val1 = np.array([3, 2, 2, 2, 1, 1, 3, 2, 1, 2, 3])
col_idx1 = np.array([1, 2, 4, 2, 3, 3, 3, 4, 5, 5, 6])
row_ptr1 = np.array([1, 4, 6, 7, 9, 10, 12])
x1 = np.array([1, 2, 3, 4, 5, 6])

val2 = np.array([1, 2, 1, 2, 3, 1 , 2])
col_idx2 = np.array([0, 2, 0, 2, 3, 1, 3])
row_ptr2 = np.array([0, 2, 2, 5, 7])
x2 = np.array([1, 2, 8, 3])

# test and verify accuracy against dense matrix-vector product
assert (sparse_matrix_vector_product(x1, val1, col_idx1, row_ptr1)).all() == (test_smvp(x1, val1, col_idx1, row_ptr1)).all()
assert (sparse_matrix_vector_product(x2, val2, col_idx2, row_ptr2)).all() == (test_smvp(x2, val2, col_idx2, row_ptr2)).all()


# Tests 2: QR factorization
A = np.array([[2, -1], [-1, 2]])
Q, R = QR_fac(A)
I = np.identity(shape(Q)[0])

A1 = np.array([[2, 3, 4], [5, 0, 1], [6, 4, 3]])
Q1, R1 = QR_fac(A1)
I1 = np.identity(shape(Q1)[0])

assert (matrix_matrix_product(Q, R)).all() == A.all() # test QR = A
assert np.allclose(R, np.triu(R)) == True             # test if R is upper triangular
assert frobenius_norms(np.transpose(Q), Q, I) == 0    # test ||Q^TQ-I||_F
assert frobenius_norms(Q, R, A) == 0                  # test ||QR-A||_F

assert (matrix_matrix_product(Q1, R1)).all() == A1.all() # test QR = A
assert np.allclose(R1, np.triu(R1)) == True              # test if R is upper triangular
assert frobenius_norms(np.transpose(Q1), Q1, I1) == 0    # test ||Q^TQ-I||_F
assert frobenius_norms(Q1, R1, A1) == 0                  # test ||QR-A||_F



# Tests 3: direct solver Ax=b, residual || Ax-b ||, and || x-y || 
A2 = np.array([[5, 6, 33, 1], 
              [34, 2, 1, 0], 
              [0, 21, 5, 67], 
              [0, 7, 4, 3]])
b2 = np.array([1, 6, 0, 99])
real_x2 = np.array([-327395/406086, 7391219/406086, -607246/203043, -2226017/406086])
x2 = direct_solver(A2, b2)

A3 = np.array([[1, -1], [0, 2]])
b3 = np.array([1, 1])
real_x3 = np.array([1.5, 0.5])
x3 = direct_solver(A3, b3)

# residual ||Ax - b||, lower than 10^(-10) is OK
assert euclidian_dist(matrix_vector_product(x2, A2), b2) <= 10**(-10) 
assert euclidian_dist(matrix_vector_product(x3, A3), b3) <= 10**(-10)

# ||x - y|| , lower than 10^(-10) is OK
assert euclidian_dist(real_x3, x3) <= 10**(-10)                     
assert euclidian_dist(real_x2, x2) <= 10**(-10) 


# **Discussion**

I thought it was interesting to see how the QR factorization can be used to solve equations on the form of Ax=b.  